In [1]:
import configparser
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
import boto3
import awswrangler

In [3]:
config = configparser.ConfigParser()
config.read_file(open('credentials.config'))

In [4]:
db = config['POSTGRES']['PG_DB']
user = config['POSTGRES']['PG_UNAME']
passwd = config['POSTGRES']['PG_PASS']
port = config['POSTGRES']['PG_PORT']
host = config['POSTGRES']['PG_HOST']

In [5]:
host

'rds-pandas-tutorial.coc5gkht2i7a.us-east-1.rds.amazonaws.com'

In [16]:
credentials = "postgresql://{}:{}@{}:{}/{}".format(user,passwd,host,port,db)

#using psycopg2 to test connection since there are no tables
import psycopg2
try:
    conn = psycopg2.connect(host=host,dbname=db,user=user,password=passwd,port=port)
except Exception as e:
    print(e)
    
conn.set_session(autocommit=True)

try:
    cur = conn.cursor()
    
except:
    print(e)

In [7]:
#Helper functions to work with the database
def schemaGen(dataframe, schemaName):
    localSchema = pd.io.sql.get_schema(dataframe,schemaName)
    localSchema = localSchema.replace('TEXT','VARCHAR(255)').replace('INTEGER','NUMERIC').replace('\n','').replace('"',"")
    return "".join(localSchema)

#Using pandas read_sql for getting schema
def getSchema(tableName, credentials):
    schema = pd.read_sql("""SELECT * FROM information_schema.columns where table_name='{}'""".format(tableName),con=credentials)
    return schema

#Issue is in using pd.read_sql to write data to the database. so using psycopg2
def queryTable(query):
    try:
        schema = cur.execute(query)
        return 
    except Exception as e:
        print(e)
        
#This doesn't return anything

#Using the pd.read_sql for getting data from db
def queryBase(query):
    requiredTable = pd.read_sql(query,con=credentials)
    return requiredTable

#This returns the dataframe

In [17]:
queryBase("""SELECT * FROM test_table""")

,identity,full_name
0,1.0,new year
1,2.0,new moon
2,3.0,new dress


In [9]:
#create the pandas dataframe
!cat source_folder/top_earners_list.csv

id,name,E-mail,Salary,occupation
1,Joel,Joel@Athena.com,187069,Mathematician
2,Afro,Afro@Glue.aws.in,752689,Physicist
3,Beatles,beatles@lambda.com,975682,Algorist
4,Snoop Dog,snoopy@apigateway.com,752689,Artificial Rapper


In [10]:
top_earners = pd.read_csv('source_folder/top_earners_list.csv')
top_earners.head()

,id,name,E-mail,Salary,occupation
0,1,Joel,Joel@Athena.com,187069,Mathematician
1,2,Afro,Afro@Glue.aws.in,752689,Physicist
2,3,Beatles,beatles@lambda.com,975682,Algorist
3,4,Snoop Dog,snoopy@apigateway.com,752689,Artificial Rapper


In [26]:
from sqlalchemy import create_engine
sqlalchemy_conn = create_engine(f"postgresql://{user}:{passwd}@{host}/{db}")

In [30]:
top_earners.to_sql(name="top_es",con=credentials,if_exists='replace',index=False)

4

In [29]:
queryBase("SELECT * FROM top_earners")

,id,name,E-mail,Salary,occupation
0,1,Joel,Joel@Athena.com,187069,Mathematician
1,2,Afro,Afro@Glue.aws.in,752689,Physicist
2,3,Beatles,beatles@lambda.com,975682,Algorist
3,4,Snoop Dog,snoopy@apigateway.com,752689,Artificial Rapper


In [ ]:
your_session = boto3.Session()